<a href="https://colab.research.google.com/github/Digx1/langgraph-notes/blob/main/module-5/memory_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core

In [2]:
import os, getpass

def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")

    # Set the environment variable for the current process
    os.environ[var] = env_value

_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

LANGCHAIN_API_KEY: ··········


In [3]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    content: str = Field(description="The main content of the memory. For example: User expressed interest in learning about French.")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

In [4]:
_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


In [5]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# Initialize the model
model = ChatOpenAI(model="gpt-4o", temperature=0)

# Bind schema to model
model_with_structure = model.with_structured_output(MemoryCollection)

# Invoke the model to produce structured output that matches the schema
memory_collection = model_with_structure.invoke([HumanMessage("My name is digvijay. I like to bike.")])
memory_collection.memories

[Memory(content="User's name is Digvijay."),
 Memory(content='User likes to bike.')]

In [6]:
memory_collection.memories[1].model_dump()

{'content': 'User likes to bike.'}

In [7]:
import uuid
from langgraph.store.memory import InMemoryStore

# Initialize the in-memory store
in_memory_store = InMemoryStore()

# Namespace for the memory to save
user_id = "1"
namespace_for_memory = (user_id, "memories")

# Save a memory to namespace as key and value
key = str(uuid.uuid4())
value = memory_collection.memories[0].model_dump()
in_memory_store.put(namespace_for_memory, key, value)

key = str(uuid.uuid4())
value = memory_collection.memories[1].model_dump()
in_memory_store.put(namespace_for_memory, key, value)

In [8]:
# Search
for m in in_memory_store.search(namespace_for_memory):
    print(m.dict())

{'namespace': ['1', 'memories'], 'key': '28bbf9a6-5683-416a-9907-69db8276308e', 'value': {'content': "User's name is Digvijay."}, 'created_at': '2025-03-18T12:18:53.592045+00:00', 'updated_at': '2025-03-18T12:18:53.592050+00:00', 'score': None}
{'namespace': ['1', 'memories'], 'key': '33a2aabe-1bd9-4265-952d-34bdb52815cd', 'value': {'content': 'User likes to bike.'}, 'created_at': '2025-03-18T12:18:53.592189+00:00', 'updated_at': '2025-03-18T12:18:53.592191+00:00', 'score': None}


In [9]:
from trustcall import create_extractor

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,
)

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Instruction
instruction = """Extract memories from the following conversation:"""

# Conversation
conversation = [HumanMessage(content="Hi, I'm digvijay."),
                AIMessage(content="Nice to meet you, digvijay."),
                HumanMessage(content="This morning I had a nice bike ride in San Francisco.")]

# Invoke the extractor
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=instruction)] + conversation})

In [11]:
result

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XzBwqEjXdRAmMhYU36MngXzD', 'function': {'arguments': '{"content":"User, Digvijay, had a nice bike ride in San Francisco this morning."}', 'name': 'Memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 114, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'id': 'chatcmpl-BCQH6NFpJs4sNIWUSv7K2Hjbh6zHC', 'finish_reason': 'stop', 'logprobs': None}, id='run-38a59651-2c46-4e7b-80e8-fcc8b8bd5304-0', tool_calls=[{'name': 'Memory', 'args': {'content': 'User, Digvijay, had a nice bike ride in San Francisco this morning.'}, 'id': 'call_XzBwqEjXdRAmMhYU36MngXzD', 'type': 'tool_call'}], usage_metad

In [18]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_2C31tAYUSe5SQe7u9Yupas2q)
 Call ID: call_2C31tAYUSe5SQe7u9Yupas2q
  Args:
    content: User, named Digvijay, had a nice bike ride in San Francisco this morning. After the ride, they went to Tartine and ate a croissant. They were also thinking about their trip to Japan and planning to go back this winter.
  Memory (call_h83wZTNEO1kRWKvGWGBbtqDj)
 Call ID: call_h83wZTNEO1kRWKvGWGBbtqDj
  Args:
    content: User went to Tartine and ate a croissant after their bike ride in San Francisco. They were also thinking about their trip to Japan and planning to go back this winter.


In [16]:
# Update the conversation
updated_conversation = [AIMessage(content="That's great, did you do after?"),
                        HumanMessage(content="I went to Tartine and ate a croissant."),
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

# Update the instruction
system_msg = """Update existing memories and create new ones based on the following conversation:"""

# We'll save existing memories, giving them an ID, key (tool name), and value
tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result["responses"] else None
existing_memories

[('0',
  'Memory',
  {'content': 'User, named Digvijay, had a nice bike ride in San Francisco this morning.'})]

In [17]:
# Invoke the extractor with our updated conversation and existing memories
result = trustcall_extractor.invoke({"messages": updated_conversation,
                                     "existing": existing_memories})

In [19]:
# Responses contain the memories that adhere to the schema
for m in result["responses"]:
    print(m)

content='User, named Digvijay, had a nice bike ride in San Francisco this morning. After the ride, they went to Tartine and ate a croissant. They were also thinking about their trip to Japan and planning to go back this winter.'
content='User went to Tartine and ate a croissant after their bike ride in San Francisco. They were also thinking about their trip to Japan and planning to go back this winter.'


In [20]:
# Metadata contains the tool call
for m in result["response_metadata"]:
    print(m)


{'id': 'call_2C31tAYUSe5SQe7u9Yupas2q', 'json_doc_id': '0'}
{'id': 'call_h83wZTNEO1kRWKvGWGBbtqDj'}
